In [7]:
# This code will convert raw data to readable dataset
import glob 
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt

# Read all csv files with same file name in the folder
filenames = sorted(glob.glob('./Desktop/19_2018-02-0*.csv'))
appended_data=[]

# Loop
for f in filenames:
     
    df = pd.read_csv(f)


    df = df.drop(columns=['X','Y','Z', 'Pa','dbm','temp','num','datetime','ID'])

    df['Date'] =  pd.to_datetime(df['Date'])+pd.Timedelta('09:00:00')

    df['stepdiff']=df['step'].diff().abs()
    df['transtotal'] = (df.stat!=df.stat.shift()).astype(int).cumsum().diff()
    df['transdiff']=df['transtotal'].diff().abs()


    data = df.loc[:, ['transdiff','stepdiff','Acc' ]]
    data = data.set_index(df.Date)

    import numpy as np

    minute = data.resample('5Min').agg({'transdiff': np.sum, 'stepdiff': np.sum, 'Acc': np.mean}).dropna()

    minute['movavgacc'] = minute['Acc'].rolling(5, min_periods=1).mean()
    minute['movavgstep'] = minute['stepdiff'].rolling(3, min_periods=1).mean()
    minute['movavgtrans'] = minute['transdiff'].rolling(3, min_periods=1).mean()
    minute['movstdacc'] = minute['Acc'].rolling(5, min_periods=1).std()
    minute['movstdstep'] = minute['stepdiff'].rolling(3, min_periods=1).std()
    minute['movstdtrans'] = minute['transdiff'].rolling(3, min_periods=1).std()

    from datetime import datetime, timedelta



    ################## SET CALVING TIME HERE ####################
    calvingtime = datetime.strptime("18/02/04 02:50", "%y/%m/%d %H:%M")

    # Define time range to make label

    # This is what to change to differ training data (hourly, daily)

    a = timedelta(minutes=180) # Set time to limit to 2 hours before calving


    # For time between 0 - 2 hours to calving time, label 1
    # Else label 0

    def label(c):
        if calvingtime - c['time'] < timedelta(days=0): 
           return '0'
        elif calvingtime - c['time'] > a:
           return '0'
        else:
           return '1'

    # Create new column delta time and label
    minute['time'] = pd.to_datetime(minute.index)
    minute['delta'] = calvingtime - minute['time']
    minute['label'] = minute.apply(label, axis=1)
    minute['hoursto'] = (minute['delta'] / np.timedelta64(1, 'h')).astype(int)

    minute = minute.ix[minute['hoursto'] >= 0]
    #minute['ttc'] = minute['delta'].dt.seconds/3600
    #minute['ttc'] = minute['ttc'].astype(int)


    df = minute

    df['index1'] = df.index.hour

    bins = [0, 5, 13, 17, 25]
    labels = ['morning','afternoon','evening','night']
    hours = df['index1']
    df['bin'] = pd.cut(hours-5+24 *(hours<5),bins=bins,labels=labels,right=False)
    df = pd.concat([df, pd.get_dummies(df['bin'])], axis=1)
    df = df.drop(columns=['time','delta','index1','bin'])
    df = df.dropna()
    appended_data.append(df)

appended_data = pd.concat(appended_data)

#appended_data.to_csv(f'{f[:-10]}_data.csv', index=True)
appended_data.to_csv('contohtrainingdata.csv', index=True)

appended_data

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:72: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,transdiff,stepdiff,Acc,movavgacc,movavgstep,movavgtrans,movstdacc,movstdstep,movstdtrans,label,hoursto,morning,afternoon,evening,night
Date,,,,,,,,,,,,,,,
2018-02-03 00:05:00,4.0,25.0,0.059393,0.043928,240.500000,23.000000,0.021871,304.763023,2.687006e+01,0,26,0,0,0,1
2018-02-03 00:10:00,48.0,716.0,0.038184,0.042013,399.000000,31.333333,0.015817,349.008596,2.386071e+01,0,26,0,0,0,1
2018-02-03 00:15:00,12.0,66.0,0.016849,0.035722,269.000000,21.333333,0.018030,387.655775,2.343786e+01,0,26,0,0,0,1
2018-02-03 00:20:00,0.0,0.0,0.001069,0.028792,260.666667,20.000000,0.021999,395.708647,2.497999e+01,0,26,0,0,0,1
2018-02-03 00:25:00,22.0,143.0,0.014101,0.025919,69.666667,11.333333,0.022969,71.570478,1.101514e+01,0,26,0,0,0,1
2018-02-03 00:30:00,28.0,264.0,0.036955,0.021432,135.666667,16.666667,0.015898,132.152689,1.474223e+01,0,26,0,0,0,1
2018-02-03 00:35:00,0.0,5.0,0.012032,0.016201,137.333333,16.666667,0.013057,129.592952,1.474223e+01,0,26,0,0,0,1
2018-02-03 00:40:00,0.0,0.0,0.026833,0.018198,89.666667,9.333333,0.013916,150.997792,1.616581e+01,0,26,0,0,0,1
2018-02-03 00:45:00,0.0,1.0,0.040109,0.026006,2.000000,0.000000,0.012811,2.645751,5.840039e-07,0,26,0,0,0,1


In [ ]:
appended_data.to